<a href="https://colab.research.google.com/github/JoaoAnjos-DataScience/JoaoAnjos-DataScience/blob/main/Modelo_de_Previs%C3%A3o_de_Comra_(Moto_ou_Carro).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Modelo de Previsão de Compra (Moto ou Carro)**



Esse código em Python lê um arquivo CSV com respostas de clientes sobre sua preferência entre moto (MT), carro (CR) ou indeciso (IN) e calcula as probabilidades de cada opção para cada cliente. Em seguida, ele adiciona a escolha mais provável à lista de probabilidades e imprime uma tabela com as probabilidades e a escolha mais provável para cada cliente. Finalmente, ele converte a lista de probabilidades em um DataFrame do Pandas e exporta para um arquivo CSV, e depois disso iremos utilizar aprendisado de máquina para prever o conjunto de dados.

In [ ]:
#Nessa primeira parte estou fazendo uma calculadora de probabilidades de compras de Carros ou Motos podendo ser modificado de acordo com a nessecidade.
#No final do Código exportaremos um novo arquivo onde deixarei já como default csv, que iremos utilizar para fazer o modelo de Machine Leanning.

import csv
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
import numpy as np

moto_count = 0
carro_count = 0
nao_sei_count = 0
current_row = 0

with open('/content/Respostas.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pular primeira linha
    probabilidades = []  # lista para armazenar as probabilidades

    for row in reader:
        escolha = row[1]
        if escolha == 'MT':
            moto_count += 1
        elif escolha == 'CR':
            carro_count += 1
        else:
            nao_sei_count += 1

        total_count = moto_count + carro_count + nao_sei_count

        prob_moto = moto_count / total_count
        prob_carro = carro_count / total_count
        prob_nao_sei = nao_sei_count / total_count

        # adicionar as probabilidades à lista do modelo( por Média)
        probabilidades.append((prob_moto, prob_carro, prob_nao_sei))

        # adicionar a escolha mais provável à lista de probabilidades (modelo)
        escolha_mais_provavel = max((prob_moto, 'moto'), (prob_carro, 'carro'), (prob_nao_sei, 'indeciso'))
        probabilidades[-1] += (escolha_mais_provavel[1],)  # adicionar a escolha à última tupla da lista



# definir a largura das colunas
col_width = 10

# imprimir o cabeçalho da tabela
print(f"{'Cliente_ID':{col_width}} {'Moto':{col_width}} {'Carro':{col_width}} {'Indeciso':{col_width}} {'Escolha Provavel':{col_width}}")

# imprimir as probabilidades
for i, prob in enumerate(probabilidades):
    escolha = f"{i+1}"
    prob_moto = f"{prob[0]:.2%}"
    prob_carro = f"{prob[1]:.2%}"
    prob_nao_sei = f"{prob[2]:.2%}"
    escolha_mais_provavel = prob[3]
    print(f"{escolha:{col_width}} {prob_moto:{col_width}} {prob_carro:{col_width}} {prob_nao_sei:{col_width}} {escolha_mais_provavel:{col_width}}")

# converter a lista de tuplas em um DataFrame
df = pd.DataFrame(probabilidades, columns=['Prob. Moto', 'Prob. Carro', 'Prob. Indeciso', 'Escolha Provavel'])
df.index.name = 'Cliente_ID'

# exportar para CSV
df.to_csv('Escolha Possivel.csv')

# Carregar os dados para modelo
df = pd.read_csv('/content/Escolha Possivel.csv')

# Criar a matriz de recursos e o vetor alvo
X = df[['Prob. Moto',	'Prob. Carro','Prob. Indeciso']]
y = df['Escolha Provavel'].apply(lambda x: 1 if x == 'carro' else 0)

# Ajustar o modelo de regressão linear
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Avaliar a precisão do modelo
accuracy_scores = []
for i in range(10):
  # Dividir em treinamento e teste
  train = df.sample(frac=0.8, random_state=i)
  test = df.drop(train.index)

  # Ajustar o modelo de regressão linear
  X_train = train[['Prob. Moto',	'Prob. Carro','Prob. Indeciso']]
  y_train = train['Escolha Provavel'].apply(lambda x: 1 if x == 'carro' else 0)
  X_train = sm.add_constant(X_train)
  model = sm.OLS(y_train, X_train).fit()

  # Fazer previsões no conjunto de teste
  X_test = test[['Prob. Moto',	'Prob. Carro','Prob. Indeciso']]
  X_test = sm.add_constant(X_test)
  y_test = test['Escolha Provavel'].apply(lambda x: 1 if x == 'carro' else 0)
  y_pred = model.predict(X_test)

